In [108]:
import glob
import os
import re
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

In [109]:
df_vids = pd.concat([pd.read_pickle(i) for i in glob.glob(
    '../pickles/individual/*full.pkl')], ignore_index=True)  # * local
df_vids.dropna(inplace=True)
# minimum number of videos before a game is considered
df_vids = df_vids.groupby("game").filter(lambda x: len(x) >= 5)
df_vids = df_vids[['channel', 'title', 'date', 'likes', 'comments', 'views',
       'game']].copy()
df_vids.rename(columns={'channel':'Channel Title', 'title':'Video Title', 'date': 'Upload Date', 'likes': 'Likes', 'comments': 'Comments', 'views':'Views',
       'game':'Game Played'}, inplace=True)
df_vids = df_vids[~df_vids['Game Played'].isin(['1996', '2000', '2004', '2009', '2010', '2011', '2012', '2013',
                                        '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', 'VRChat', 'Youtubers Life'])].copy()
df_vids = df_vids.astype({'Likes':'int', 'Comments':'int', 'Views':'int'})
df_vids['Engagement'] = df_vids['Likes'] + df_vids['Comments']
df_vids.drop(columns=['Likes', 'Comments'])
df_vids['Upload Date'] = pd.to_datetime(df_vids['Upload Date'], errors='coerce').dt.tz_localize(None)
df_vids.replace('SeaNanners Gaming Channel', 'SeaNanners', inplace=True)
df_vids.replace('Splattercatgaming', 'Splattercat', inplace=True)
# df_vids.to_pickle('./dfvids.p')
df_vids.sample(10)
pd.concat()

,Channel Title,Video Title,Upload Date,Likes,Comments,Views,Game Played,Engagement
38467,Many A True Nerd,Dragon Age: Origins - Part 14 - The Monster Mash,2021-10-22 21:08:45,2340,379,50155,Dragon Age: Origins,2719
36746,Mathas,LETS PLAY PROJECT ZOMBOID | EPISODE 21,2014-04-12 13:00:04,198,23,7881,Project Zomboid,221
5238,Blitz,Escapists 2 Versus Mode! - Baron Vs Blitz Epic...,2017-08-14 16:04:08,5646,251,524572,The Escapists 2,5897
14320,H2ODelirious,Among The Sleep Ep.4 (Did we find Mommy?) Last...,2014-06-08 22:28:28,106269,8400,5150539,Among the Sleep,114669
60310,PartyElite,IXION | EP.19 - WEATHERING THE STORM (NEW Sci-...,2023-01-14 14:00:20,173,31,2514,IXION,204
2264,AngryJoeShow,AngryJoe Plays Farcry 4! [Arena Mode],2014-11-15 23:06:49,7232,2600,493581,Far Cry 4,9832
90402,Splattercat,Ghost Recon Wildlands - Fiesta Pantalones - Ep...,2017-03-04 16:00:09,903,87,31138,Tom Clancy's Ghost Recon Wildlands,990
29033,Markiplier,Drunk Minecraft #59 | THE WALKING DEAD,2013-12-10 18:42:25,24622,2100,1937311,Minecraft,26722
61105,PartyElite,Total War: Warhammer 2 | The Twisted & The Twi...,2020-11-19 15:22:50,593,126,10117,Total War: WARHAMMER II,719
59514,Northernlion,Let's Play - The Binding of Isaac - Episode 24...,2012-07-14 14:00:09,1238,582,69925,The Binding of Isaac,1820


In [110]:
# # total views
# tviews = df_vids.groupby(by=['Channel Title'])["Genres"].nunique()
# # x, y = tviews.index, tviews.values
# # tviews.merge(df_vids['Channel Title, '])
# # a = df_vids.groupby(by=['Channel Title']).Views.sum()/df_vids.groupby(by=['Channel Title']).Views.count()
# # a = a.reset_index()
# # a.sort_values(by=['Views'], ascending=False)
# tviews

In [129]:
# tviews.plot.hist()
for i in df_vids[df_vids['Channel Title'] == 'Markiplier'].groupby(by=['Game Played']).Views.sum().sort_values(ascending=False).head(10).index:
    print(i)




Five Nights at Freddy's
Garry's Mod
Minecraft
Five Nights at Freddy's: Sister Location
Subnautica
Five Nights at Freddy's 4
The Forest
Yandere Simulator
Amnesia: The Dark Descent
Five Nights at Freddy's 3
Slender: The Arrival
Five Nights at Freddy's 2
Freddy Fazbear's Pizzeria Simulator
Raft
Getting Over It with Bennett Foddy
Phasmophobia
Turbo Dismount
Resident Evil Village
West of Loathing
7 Days to Die


In [112]:
df_games = pd.read_pickle('../pickles/df_games.pkl')
df_games = df_games[['name', 'count', 'metacritic', 'price',
       'releaseDate', 'desc', 'header_image', 'genres',
       'categories']].copy()
df_games.rename(columns={'name':'Game Title', 'count':'Video Count', 'metacritic':'Metacritic Score', 'price':'Price (USD)',
       'releaseDate':'Release date (steam)', 'desc':'Description', 'genres':'Genres',
       'categories':'Tags'}, inplace=True)
df_games['Price (USD)'] = df_games['Price (USD)']/100
df_games['Release date (steam)'] = pd.to_datetime(df_games['Release date (steam)'], errors='coerce')
df_games['Price (USD)'][df_games['Price (USD)'] >= 120] = np.nan

C:\Users\crystallikelaw\AppData\Local\Temp\ipykernel_14876\32984481.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games['Price (USD)'][df_games['Price (USD)'] >= 120] = np.nan


In [113]:
# Cleaning functions
def cleanHtml(text: str) -> str:
    """
    Removes HTML symbols from text

    Args:
        text (str): input text

    Returns:
        str: text without things like %2E in it
    """
    html = re.compile(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});")
    return re.sub(html, r' ', text)


def cleanUrl(text: str) -> str:
    """
    Removes http(s) urls from text

    Args:
        text (str): input text

    Returns:
        str: output text without urls like https://google.com/
    """
    return re.sub(r"https?://\S+|www\.\S+", r' ', text)


def cleanAscii(text: str) -> str:
    """
    Remove non ASCII characters

    Args:
        text (str): input text

    Returns:
        str: text without things like carriage returns, tabs, null bytes
             removes from \x00 (null byte) to \x7f (delete)
    """
    return re.sub(r'[^\x00-\x7f]', r' ', text)


def cleanSpecChar(text: str) -> str:
    """
    Removes misc special unicode chars

    Args:
        text (str): input text

    Returns:
        str: text without emojis, pictographs, maps, flags.. 
    """
    specPattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    return specPattern.sub(r' ', text)


def cleanSpaces(text: str) -> str:
    """
    Converts multiple spaces in text to a single space

    Args:
        text (str): input text

    Returns:
        str: text with at most contiguous space
    """
    return re.sub(r'\s{2,}', r' ', text)


def fullClean(text: str) -> str:
    """
    Compiles the above cleaning functions

    Args:
        text (str): input text

    Returns:
        str: cleaned output text, lowercase
    """
    return cleanSpaces(cleanSpecChar(cleanAscii(cleanHtml(cleanUrl(text))))).strip()

df_games['Description'] = df_games['Description'].progress_apply(fullClean)

100%|██████████| 686/686 [00:00<00:00, 6783.54it/s]


In [114]:
df_games.sample(10)

,Game Title,Video Count,Metacritic Score,Price (USD),Release date (steam),Description,header_image,Genres,Tags
24,The Long Dark,340,NaN,19.99,2017-08-01,THE LONG DARK: SURVIVAL EDITION - This edition...,https://cdn.akamai.steamstatic.com/steam/apps/...,"[Adventure, Indie, Simulation, Strategy]","[Single-player, Steam Achievements, Full contr..."
576,A House of Many Doors,13,NaN,10.99,2017-02-03,Welcome to the House. You are not welcome. Exp...,https://cdn.akamai.steamstatic.com/steam/apps/...,"[Indie, RPG]",[Single-player]
547,The Dark Pictures Anthology: The Devil in Me,14,None,39.99,2022-11-17,A series of stand-alone branching cinematic ho...,https://cdn.akamai.steamstatic.com/steam/apps/...,[Adventure],"[Single-player, Multi-player, Co-op, Online Co..."
8,Civilization V,890,90.0,29.99,2010-09-21,The Flagship Turn-Based Strategy Game Returns ...,https://cdn.akamai.steamstatic.com/steam/apps/...,[Strategy],"[Single-player, Multi-player, Steam Achievemen..."
672,Pony Island,10,86.0,4.99,2016-01-04,Pony Island is a suspense puzzle game in disgu...,https://cdn.akamai.steamstatic.com/steam/apps/...,[Indie],"[Single-player, Steam Achievements, Steam Trad..."
631,Forsaken Isle,11,NaN,7.99,2015-02-06,Forsaken Isle is a wilderness survival game fo...,https://cdn.akamai.steamstatic.com/steam/apps/...,"[Action, Adventure, Casual, Indie, RPG]",[Single-player]
455,Borderlands,20,81.0,59.99,2020-03-13,"The original shooter-looter returns, packing b...",https://cdn.akamai.steamstatic.com/steam/apps/...,"[Action, RPG]","[Single-player, Multi-player, Co-op, Online Co..."
325,Inscryption,29,NaN,19.99,2021-10-19,From the creator of Pony Island and The Hex co...,https://cdn.akamai.steamstatic.com/steam/apps/...,"[Adventure, Indie, Strategy]","[Single-player, Steam Achievements, Steam Trad..."
218,Wolfenstein: The New Order,54,81.0,19.99,2014-05-19,Wolfenstein : The New Order reignites the seri...,https://cdn.akamai.steamstatic.com/steam/apps/...,[Action],"[Single-player, Steam Achievements, Partial Co..."
529,Sunset Overdrive,15,83.0,22.99,2018-11-16,Don t miss the single-player campaign from the...,https://cdn.akamai.steamstatic.com/steam/apps/...,"[Action, Adventure]","[Single-player, Steam Achievements, Full contr..."


In [115]:
df_games.to_pickle('./dfgames.p')
x = df_games.Genres.sample().to_list()[0]
x

['Action', 'Adventure', 'RPG']

In [116]:
df_vids = df_vids.merge(df_games[['Game Title', 'Genres']], left_on="Game Played", right_on="Game Title", how='inner')
df_vids.drop(columns=['Game Title'], inplace=True)
df_vids['Upload Year'] = df_vids['Upload Date'].dt.year
df_vids.to_pickle('./dfvids.p')
df_vids.sample(5)

,Channel Title,Video Title,Upload Date,Likes,Comments,Views,Game Played,Engagement,Genres,Upload Year
4442,AngryJoeShow,Wildermyth - Angry Impressions [Great D&D+RPG+...,2021-07-10 20:05:25,3001,232,82167,Wildermyth,3233,"[Indie, RPG, Strategy]",2021
52934,Northernlion,Northernlion Plays: Europa Universalis IV! [Ep...,2013-08-30 18:00:46,703,72,13157,Europa Universalis IV,775,"[Simulation, Strategy]",2013
26578,Mathas,Don't Starve Together - Episode 38 - WE'RE BAA...,2015-03-24 15:00:03,676,35,32261,Don't Starve,711,"[Adventure, Indie, Simulation]",2015
46786,Northernlion,Let's Play - VVVVVV - [4] Vitellary,2011-01-01 05:53:28,237,46,23376,VVVVVV,283,"[Action, Indie]",2011
45830,Northernlion,Let's Play - The Binding of Isaac - Episode 55...,2013-06-28 16:00:42,1435,407,52891,The Binding of Isaac,1842,"[Action, Adventure, Indie, RPG]",2013


In [131]:
import sklearn

In [132]:
sklearn.__version__

'1.2.0'